In [4]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import image_classification
from aiy.vision.streaming.server import StreamingServer
from aiy.vision.streaming import svg
from aiy.leds import Leds, Color
from aiy.board import Board

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import contextlib
import time

### Image classifier

Remember if your joy detector is running, you need to turn it off using the commands

```
sudo systemctl stop joy_detection_demo.service
```

I added some code here that is called to create the overlay - basically the box around the objects and the labels above. You can customize it and/or add information you want to overlay on the camera feed.

In [22]:
def svg_overlay(classes, frame_size):

    width, height = frame_size
    doc = svg.Svg(width=width, height=height)
    doc.add(svg.Text(classes_info(classes), x=50, y=50, fill='white', font_size=50))
    
    return str(doc)

In [23]:
def classes_info(classes):
    return '\n'.join('%s (%.2f)' % pair for pair in classes)

def classes_process(classes, threshold):
    return [pair for pair in classes if pair[1] > threshold]

#### Main loop

Here is our main loop. To watch the camera feed, you can connect to http://orcspi-vis.local:4664.

In [ ]:
detection_threshhold = 0.5      # Confidence thresshold to list an object

with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))
    board  = stack.enter_context(Board())

    # This starts and runs the streaming of the camera
    server = stack.enter_context(StreamingServer(camera))  

    # Sets what should happen when button is pressed
   # board.button.when_pressed = match_reset
    
    print ("Loading model - hold on ..")
    
    
    # Do inference on VisionBonnet
    with CameraInference(image_classification.model()) as inference:
        
        try:   
            for result in inference.run():
                leds.update(Leds.rgb_on(Color.RED))
                classes = image_classification.get_classes(result, top_k=5)
                topclasses = classes_process(classes, detection_threshhold)

                if(len(topclasses)>0):
                    clear_output(wait=True)
                    leds.update(Leds.rgb_on(Color.BLUE))
                    print(classes_info(topclasses))
                    server.send_overlay(svg_overlay(topclasses, (result.width, result.height)))
                                                                                 
        except KeyboardInterrupt:
            print("Interrupted ..")
            
    leds.update(Leds.rgb_off())    
    print("Done")

crossword puzzle/crossword (0.51)
